In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from vaastav.understat import *
from vaastav.getters import *
from config import *

In [3]:
fixtures = get_fixtures_data()

In [4]:
df_fixtures = pd.DataFrame(fixtures).dropna()

In [5]:
def team_id_to_name(current_teams, team_id):
    return current_teams[team_id - 1]

In [6]:
df_fixtures["Squad H"] = df_fixtures["team_h"].apply(lambda x: team_id_to_name(current_teams, x))
df_fixtures["Squad A"] = df_fixtures["team_a"].apply(lambda x: team_id_to_name(current_teams, x))
df_fixtures["date"] = df_fixtures["kickoff_time"].str[:10]

In [7]:
df_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,Squad H,Squad A,date
2,2367538,1.0,True,True,1,2023-08-11T19:00:00Z,90,False,True,13,3.0,6,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,93321,Burnley,Manchester City,2023-08-11
3,2367540,1.0,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,93322,Arsenal,Nott'ham Forest,2023-08-12
4,2367539,1.0,True,True,3,2023-08-12T14:00:00Z,90,False,True,19,1.0,3,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93323,Bournemouth,West Ham,2023-08-12
5,2367541,1.0,True,True,4,2023-08-12T14:00:00Z,90,False,True,12,1.0,5,4.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,3,93324,Brighton,Luton,2023-08-12
6,2367542,1.0,True,True,5,2023-08-12T14:00:00Z,90,False,True,10,1.0,9,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93325,Everton,Fulham,2023-08-12
7,2367544,1.0,True,True,7,2023-08-12T14:00:00Z,90,False,True,8,1.0,17,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93327,Sheffield United,Crystal Palace,2023-08-12
8,2367543,1.0,True,True,6,2023-08-12T16:30:00Z,90,False,True,2,1.0,15,5.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,4,93326,Newcastle Utd,Aston Villa,2023-08-12
9,2367545,1.0,True,True,8,2023-08-13T13:00:00Z,90,False,True,18,2.0,4,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,3,93328,Brentford,Tottenham,2023-08-13
10,2367546,1.0,True,True,9,2023-08-13T15:30:00Z,90,False,True,11,1.0,7,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,3,93329,Chelsea,Liverpool,2023-08-13
11,2367547,1.0,True,True,10,2023-08-14T19:00:00Z,90,False,True,20,0.0,14,1.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,4,93330,Manchester Utd,Wolves,2023-08-14


In [8]:
epl_data = get_epl_data()
epl_data = epl_data[0]

In [9]:
# epl_data

In [10]:
def team_understat_to_fpl(team_name):
    if team_name == "Wolverhampton Wanderers":
        return "Wolves"
    if team_name == "Newcastle United":
        return "Newcastle Utd"
    if team_name == "Manchester United":
        return "Manchester Utd"
    if team_name == "Nottingham Forest":
        return "Nott'ham Forest"
    return team_name

In [11]:
df = pd.DataFrame(columns=["Team", "xG", "xGA", "Date"])

for k in epl_data.keys():
    title = team_understat_to_fpl(epl_data[k]['title'])
    for h in epl_data[k]['history']:
        xG = h["xG"]
        xGA = h["xGA"]
        date = h["date"][:10]
        
        new_row = {
                    "Team": title,
                    "xG": xG,
                    "xGA": xGA,
                    "Date": date
                  }
        df = df.append(new_row, ignore_index=True)

In [12]:
def assign_opponent(x):
    r1 = df_fixtures[ (df_fixtures["date"] == x["Date"]) & (df_fixtures["Squad H"] == x["Team"]) ]
    r2 = df_fixtures[ (df_fixtures["date"] == x["Date"]) & (df_fixtures["Squad A"] == x["Team"]) ]
    if r1.size > 0:
        return r1["Squad A"].item()
    if r2.size > 0:
        return r2["Squad H"].item()
    return None

In [13]:
# df_fixtures

In [14]:
# df_fixtures["Squad A"].unique()

In [15]:
# df["Team"].unique()

In [16]:
for i, row in df.iterrows():
    df.loc[i, "Opponent"] = assign_opponent(row)

In [17]:
summary_df = df.groupby('Team', as_index=False)['xG', 'xGA'].sum()
summary_df_per_90 = df.groupby('Team', as_index=False)['xG', 'xGA'].mean()
# Add a summary row for each unique 'Team'
# summary_df.append({'Team': 'Summary', 'xG': summary_df['xG'].sum(), 'xGA': summary_df['xGA'].sum()}, ignore_index=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15780\2432573045.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary_df = df.groupby('Team', as_index=False)['xG', 'xGA'].sum()
C:\Users\user\AppData\Local\Temp\ipykernel_15780\2432573045.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary_df_per_90 = df.groupby('Team', as_index=False)['xG', 'xGA'].mean()


In [18]:
summary_df["Opponent"] = "Summary"
summary_df_per_90["Opponent"] = "Summary per 90"

In [19]:
# summary_df

In [20]:
df = df.append(summary_df)
df = df.append(summary_df_per_90)
df = df.reset_index()

In [21]:
df.tail(40)

,index,Team,xG,xGA,Date,Opponent
56,0,Arsenal,6.279430,2.885282,NaN,Summary
57,1,Aston Villa,7.566270,5.674150,NaN,Summary
58,2,Bournemouth,3.380532,6.999030,NaN,Summary
59,3,Brentford,6.324451,2.913669,NaN,Summary
60,4,Brighton,8.897400,7.526850,NaN,Summary
61,5,Burnley,0.941637,5.237650,NaN,Summary
62,6,Chelsea,6.955930,3.478021,NaN,Summary
63,7,Crystal Palace,4.405930,3.576645,NaN,Summary
64,8,Everton,5.209395,6.227080,NaN,Summary
65,9,Fulham,2.831156,9.289770,NaN,Summary


In [22]:
# assigning weighted
for i, row in df[~df["Date"].isna()].iterrows():
    # print(df[(df["Team"] == row["Opponent"]) & (df["Opponent"] == "Summary per 90")])
    df.loc[i, "Weighted_xG"] = row["xG"] / df[(df["Team"] == row["Opponent"]) & (df["Opponent"] == "Summary per 90")]["xGA"].item()
    df.loc[i, "Weighted_xGA"] = row["xGA"] / df[(df["Team"] == row["Opponent"]) & (df["Opponent"] == "Summary per 90")]["xG"].item()

In [23]:
df.head(50)

,index,Team,xG,xGA,Date,Opponent,Weighted_xG,Weighted_xGA
0,0,Aston Villa,1.486000,4.322080,2023-08-12,Newcastle Utd,1.096161,1.887687
1,1,Aston Villa,3.243360,0.721465,2023-08-20,Everton,1.562543,0.415479
2,2,Aston Villa,2.836910,0.630605,2023-08-27,Burnley,1.083276,1.339380
3,3,Everton,2.590010,1.581440,2023-08-12,Fulham,0.836407,1.675754
4,4,Everton,0.721465,3.243360,2023-08-20,Aston Villa,0.381448,1.285981
5,5,Everton,1.897920,1.402280,2023-08-26,Wolves,0.810568,0.723724
6,6,Bournemouth,1.510250,1.483400,2023-08-12,West Ham,1.008243,0.723793
7,7,Bournemouth,1.386700,3.254480,2023-08-19,Liverpool,0.835543,1.429152
8,8,Bournemouth,0.483582,2.261150,2023-08-26,Tottenham,0.299916,1.084771
9,9,Crystal Palace,2.276160,0.422704,2023-08-12,Sheffield United,0.910949,0.730490


In [24]:
summary_weighted_df = df.groupby('Team', as_index=False)['Weighted_xG', "Weighted_xGA"].mean()

C:\Users\user\AppData\Local\Temp\ipykernel_15780\1221973474.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary_weighted_df = df.groupby('Team', as_index=False)['Weighted_xG', "Weighted_xGA"].mean()


In [25]:
summary_weighted_df.sort_values(by=["Weighted_xG"], ascending=False)

,Team,Weighted_xG,Weighted_xGA
13,Manchester Utd,1.471062,0.993473
14,Newcastle Utd,1.432503,0.540503
17,Tottenham,1.308055,0.773216
10,Liverpool,1.258123,1.032051
1,Aston Villa,1.247327,1.214182
0,Arsenal,1.155114,0.835786
4,Brighton,1.129232,1.477338
12,Manchester City,1.120199,0.696725
18,West Ham,1.096564,0.832546
6,Chelsea,1.038743,0.600813


In [26]:
# summary_weighted_df["Weighted_xG"].sum() / 20